# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, WT, Male, VISl, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISp and VISl sessions. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('wt/wt') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISl' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."


In [7]:
session_id = filtered_sessions.index.values[3]

session = cache.get_session_data(session_id)

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

Total number of units: 893


,snr,waveform_recovery_slope,cumulative_drift,waveform_halfwidth,cluster_id,amplitude_cutoff,L_ratio,isi_violations,d_prime,waveform_velocity_below,waveform_spread,waveform_repolarization_slope,waveform_amplitude,peak_channel_id,waveform_PT_ratio,local_index_unit,silhouette_score,presence_ratio,waveform_velocity_above,firing_rate,isolation_distance,max_drift,nn_miss_rate,waveform_duration,nn_hit_rate,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950920817,3.246479,-0.393127,346.73,0.068677,6,0.061671,0.000078,0.061295,6.236154,0.000000,70.0,0.184384,173.942145,850186862,1.194054,6,0.148733,0.99,-0.274707,20.378176,129.193512,17.98,0.000699,0.109883,0.998000,NaN,300.0,14.019476,0.857436,0.879457,1.420513,1.288028,0.179941,0.098358,0.095574,0.013917,13.333,0.884362,0.911,0.32,8.0,NaN,0.537313,-0.238342,NaN,NaN,225.0,150.0,NaN,0.004315,0.191413,NaN,NaN,-23.333,4996,0.00,18.013558,3.623646,32.136496,3.592125,34.757226,NaN,0.0225,False,False,False,False,False,-0.136017,False,0.040639,0.042337,0.001979,0.073979,0.002895,200,59,769322712,18,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970281,1249.998762,True
950920914,3.409441,-0.089997,123.51,0.123618,11,0.000346,0.000037,0.003499,7.759853,-0.412060,80.0,0.619365,144.634425,850186876,0.410837,11,0.163029,0.99,0.000000,24.031002,100.752378,28.54,0.000000,0.247236,0.998000,NaN,200.0,1.125651,1.085897,1.530000,1.012698,1.492711,0.370398,0.011217,0.003784,0.021493,10.000,6.905333,0.397,0.32,4.0,NaN,0.067568,-0.194268,NaN,-0.093960,135.0,60.0,NaN,0.437496,0.085780,NaN,0.468639,45.000,4928,0.50,24.884868,26.484588,23.120839,26.251242,23.693264,NaN,0.2155,False,False,False,False,False,-0.033136,False,0.001183,0.004274,0.003128,0.008312,0.004566,260,11,769322712,25,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970281,1249.998762,True
950920977,5.953624,-0.034566,483.50,0.178559,14,0.000270,0.000021,0.000000,5.608442,0.412060,50.0,0.753621,282.601215,850186882,0.827002,14,0.334548,0.98,-2.747069,1.422358,79.773528,26.22,0.000101,0.370854,0.991597,NaN,100.0,12.794350,1.762424,4.421538,11.308333,7.829817,0.735024,0.320987,0.231522,0.045168,10.000,0.997451,0.217,0.02,15.0,-1.000000,-1.000000,-1.000000,-1.0,NaN,270.0,90.0,0.009782,0.015274,0.001314,0.272069,NaN,-30.000,4924,0.25,1.051585,0.452956,1.376328,0.495617,1.953678,NaN,0.2255,False,False,False,False,False,0.390559,False,0.446530,0.159236,0.197305,0.678174,0.151251,300,43,769322712,28,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970281,1249.998762,True
950921087,4.651214,-0.159289,87.94,0.109883,20,0.000133,0.000053,0.004755,8.108768,0.000000,70.0,0.862546,200.558670,850186896,0.340643,20,0.185451,0.99,0.343384,31.314676,147.363079,23.49,0.000747,0.206030,1.000000,NaN,600.0,0.577732,0.795444,0.585339,1.668627,0.782749,0.156185,0.007879,0.010648,0.008557,80.000,0.850903,0.695,0.08,8.0,NaN,-0.128114,-0.106599,NaN,-0.151399,225.0,30.0,NaN,0.035289,0.040818,NaN,0.134065,5.000,4982,0.50,30.

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISl') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 32


,snr,waveform_recovery_slope,cumulative_drift,waveform_halfwidth,cluster_id,amplitude_cutoff,L_ratio,isi_violations,d_prime,waveform_velocity_below,waveform_spread,waveform_repolarization_slope,waveform_amplitude,peak_channel_id,waveform_PT_ratio,local_index_unit,silhouette_score,presence_ratio,waveform_velocity_above,firing_rate,isolation_distance,max_drift,nn_miss_rate,waveform_duration,nn_hit_rate,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950956933,1.162808,-0.006852,112.88,NaN,452,0.004636,0.011187,0.019050,4.085391,-0.206030,70.0,0.016435,99.338460,850181612,1.753257,443,0.147069,0.99,2.541039,5.888589,86.827817,48.12,0.026134,1.346064,0.946000,NaN,1000.0,2.769629,0.755706,2.191667,1.801550,0.808513,0.338199,0.054462,0.297725,0.236882,35.000,0.643083,0.000,0.08,2.0,NaN,-1.000000,-0.054945,NaN,NaN,0.0,0.0,NaN,1.820536e-13,8.650656e-01,NaN,NaN,42.000,4943,0.50,9.508215,2.744379,7.066362,6.197403,6.541792,NaN,0.1375,False,False,False,False,False,0.359390,False,0.163844,0.053037,0.237271,0.439209,0.178563,2860,43,769322718,284,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.92566,1249.996902,True
950957006,1.736383,-0.005244,167.90,1.043886,459,0.004349,0.000116,0.085048,5.419915,-0.961474,70.0,0.048883,156.820755,850181620,1.041179,450,0.166558,0.99,-0.343384,0.675932,72.985375,40.37,0.000101,1.263652,0.990476,NaN,900.0,2.266667,1.596863,1.302523,1.390476,1.256522,1.509213,0.037443,0.249135,0.419052,65.556,0.389686,0.036,0.02,2.0,NaN,-1.000000,-0.918182,-1.0,-0.562500,0.0,0.0,NaN,1.753319e-03,5.633843e-07,0.046976,0.308545,41.111,5003,0.75,0.579801,0.612822,0.697232,0.336625,0.786667,NaN,0.1175,False,False,False,False,False,0.859373,False,0.293920,0.372308,0.599165,0.569953,0.245774,2900,43,769322718,288,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.92566,1249.996902,True
950957066,2.719674,-0.073225,84.58,0.178559,464,0.001285,0.000064,0.000000,5.951392,-0.343384,50.0,0.660438,217.212450,850181628,0.435503,455,0.210072,0.99,0.000000,2.757843,98.521667,43.22,0.000103,0.714238,0.998737,NaN,1200.0,4.270320,0.998411,1.494286,2.260606,3.833333,1.368892,0.063440,0.063342,0.217032,75.000,2.085864,0.188,0.32,2.0,NaN,-0.433962,-0.666667,NaN,-1.000000,315.0,0.0,NaN,1.137873e-01,1.332292e-02,NaN,0.000401,20.000,4989,0.25,1.373521,12.203160,3.326967,1.937511,2.573154,NaN,0.0685,False,False,False,False,False,0.558610,False,0.251448,0.639486,0.185331,0.247194,0.114628,2940,43,769322718,292,409.0,VISl,NaN,NaN,NaN,probeD,See electrode locations,29999.92566,1249.996902,True
950959834,0.390230,-0.035477,340.96,0.906533,694,0.023388,0.005460,0.093538,2.167951,0.343384,80.0,0.157112,96.800535,850181662,0.275048,683,-0.024025,0.99,0.784877,2.323878,57.416178,33.30,0.000692,0.727973,0.952770,NaN,900.0,10.375853,0.330000,1.545854,0.831008,1.225714,0.982791,0.060518,0.071252,0.134407,20.000,0.785390,0.000,0.04,1.0,NaN,-0.284672,-0.900000,NaN,-0.307692,225.0,30.0,NaN,8.448348e-02,1.155561e-04,NaN,0.6897

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
950956933     6.541792
950957006     0.786667
950957066     2.573154
950959834     0.662106
950957534     0.347705
950957567     2.848920
950957600     3.989288
950957710     0.825301
950957763     0.821304
950957743     8.384231
950957732     2.321367
950957721     7.257186
950957841     6.491168
950957796     1.727204
950957785     8.089814
950957870     1.002484
950959909     0.949196
950957886     2.754334
950957829     9.724430
950957966     1.159018
950957902     6.715645
950958066     0.518894
950958044     2.849586
950957975     1.118386
950958128     0.188507
950960188     1.276252
950958075     0.252453
950958169    16.343490
950958143    10.417176
950958253     0.817974
950958378     8.026534
950958367    16.799770
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

4.205666770647477


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
950956933     9.508215
950957006     0.579801
950957066     1.373521
950959834     2.437284
950957534     0.245158
950957567     1.305215
950957600     5.477668
950957710     1.463006
950957763     2.710506
950957743     1.197197
950957732     3.255890
950957721    22.751516
950957841     6.858072
950957796     3.476691
950957785     3.595828
950957870     1.460888
950959909     0.688878
950957886     3.236828
950957829     2.950369
950957966     1.718754
950957902     6.689691
950958066     0.915504
950958044     7.246195
950957975     2.042807
950958128     1.001812
950960188     1.449768
950958075     0.839786
950958169    11.296333
950958143     6.868132
950958253     0.488198
950958378     4.766551
950958367    10.011239
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

4.05960315349651


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])